In [4]:
import pandas as pd
import time
from collections import defaultdict
from itertools import product
import pickle

SHW = 100000

In [5]:
# g = pd.read_csv("data/attendances_over_time.csv.gz", encoding='latin-1', 
#                 header=None, names=["cust_id","age_gr","mtype_sec", "n_attend"])
# g.head()

In [6]:
mtypes_over_years = (pd.read_csv("data/ooo.csv.gz", encoding='latin-1', sep='\t')
                    .applymap(lambda x: x.split(",") if isinstance(x,str) else x)
                    )

In [7]:
mtypes_over_years.head()

,cust_id,00-17,18-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-84
0,11171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[RUNION],NaN,NaN,NaN,NaN,NaN
1,11284,NaN,NaN,NaN,NaN,NaN,[HIPHOP],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11228,NaN,NaN,NaN,NaN,NaN,"[ALTERN, LEAGUE]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[RUNION, LEAGUE]",[LEAGUE],NaN,NaN,NaN,NaN
4,11197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[RUNION],NaN,NaN,NaN,NaN,NaN


In [8]:
v = mtypes_over_years.iloc[4000000:5000000,:].T

In [9]:
v.shape

(14, 1000000)

In [10]:
v.head()

,4000000,4000001,4000002,4000003,4000004,4000005,4000006,4000007,4000008,4000009,...,4999990,4999991,4999992,4999993,4999994,4999995,4999996,4999997,4999998,4999999
cust_id,14850610,15405581,16807694,15107927,14211557,14850633,14344526,14344541,16807711,15107978,...,24042981,18253918,18229238,21479638,24042996,19716761,18253943,18229240,21479655,24043245
00-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18-24,NaN,NaN,NaN,NaN,NaN,NaN,[AFL],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25-29,"[TENNIS, CRICKET]",NaN,NaN,[BMX],NaN,NaN,"[HIPHOP, CRICKET, POP]",NaN,NaN,NaN,...,NaN,[AFL],"[RUNION, RAP]",NaN,NaN,[COUNTRY],[AFL],NaN,NaN,NaN
30-34,NaN,NaN,NaN,NaN,"[80SROCK, COUNTRY, ROCK, POP]",NaN,NaN,"[80SROCK, ROCK]","[STANDUPC, KIDLIVE, 70SPOP]",NaN,...,NaN,NaN,NaN,[70SROCK],NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
def _cust_features(c):
    
    f = defaultdict(int)
    
    s = v.ix[:,c][v[c].notnull()]
    cid = s[0]
    
    if len(s) == 2:
        f = {"age_gr": s.index[1], "@"+s[1][0]: 1}
    elif len(s) == 3:
        f = {"age_gr": s.index[-1]}
        f.update({"@"+"->".join(tp): 1 for tp in product(s[1], s[2])})
    
    yield f

f = defaultdict(lambda: defaultdict(int))  # {cust_id: {"AFL->BALLET": 1, "TENNIS->COMEDY": 1}}

print("extracting customer features...")

t1 = time.time()

for c in range(4000000,5000000):
    for fch in _cust_features(c):
        f[v.ix[:,c][0]].update(fch)
        if (c+1)%SHW == 0:
            print("extracted features for {} customers...".format(c+1))

print("done. elapsed time {:.0f} min {:.0f} sec".format(*divmod(time.time() - t1, 60)))

pd.DataFrame.from_dict(f, orient="index").to_csv("p4.csv.gz",index=False, compression="gzip")
print("saved data")

extracting customer features...
extracted features for 4100000 customers...
extracted features for 4200000 customers...
extracted features for 4300000 customers...
extracted features for 4400000 customers...
extracted features for 4500000 customers...
extracted features for 4600000 customers...
extracted features for 4700000 customers...
extracted features for 4800000 customers...
extracted features for 4900000 customers...
extracted features for 5000000 customers...
done. elapsed time 19 min 22 sec
saved data
